In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

strcutrue of our table

In [2]:
df = pd.DataFrame(columns=[
    'City', 'Interval', 'LCR', 'PGR', 'LCRPGR',
    'City Definition data', 'Built-up data', 'Population data',
    'Within city definition T1 Built-up (km²)', 'Within city definition T1 Population (*10,000)','Within city definition T1 LCPC (m²/person)',
    'Within city definition T2 Built-up (km²)', 'Within city definition T2 Population (*10,000)','Within city definition T2 LCPC (m²/person)',
    '% Change in LCPC LCPC (m²/person)', 'Built-up T1', 'Built-up T2'
])

open the brower with the SDG calculator application first, and then choose the year of T1 manually.

In [4]:
save_file_path='sdg_results_citylevel.xlsx'

In [5]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Now you can use the driver to interact with the web page
driver.get("https://eo4sdg11.users.earthengine.app/view/sdg-1131-indicator")

In [6]:
# Get the handle of the original window
original_window_handle = driver.current_window_handle
print(original_window_handle)

BDBB4DBE1B7667A7CC56D2384F9F63FB


City:
select city: we run throught all 45 cities in the list  
admin level: we use the default level, which is "Province/Country/State level"

Population: fowolling the instruction  
Data:GHS-Pop 1km as default  
T2: 2015  
T1: 1990  

Adjust threshold: default

Calculating SDG 11.3.1  
Built-up: GHS-Built 38m  
population: GHS-POP 250m

get the city list from the application. the list contains 45 cities so far.

In [18]:
driver.switch_to.window(original_window_handle)
##############################################################
# City data
##############################################################
# Find the element using its CSS selector
city_element = driver.find_elements(By.CSS_SELECTOR, '#\:3y')[0]
# print(city_element.get_attribute("aria-posinset"))
# # Use JavaScript to modify the aria-posinset attribute value of the element
# new_posinset_value = "Beijing"  # Replace with the desired new value
# driver.execute_script("arguments[0].innerText = arguments[1];", city_element, new_posinset_value)
city_element.click()
city_options = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content'))
)

In [19]:
print('number of cities:',len(city_options))

number of cities: 48


In [16]:
for city in city_options:
  print(city.text)



Stockholm
Gothenburg
Abu Dhabi
Accra
Batam
Beijing
Buenos Aires
Cairo
Charleston
Dar Es Salaam
Detroit
Dodoma
Dubai
Gombe
Guangzhou
Halle
Heidelberg
Islamabad
Johannesburg
Kampala
Karachi
Katowice
Kigali
Kitchener
Kolkata
Lagos
La Paz
Lilongwe
Luanda
Manama
Mexico City
Mogadishu
Mumbai
Nairobi
New York
Niamey
Nouakchott
Ouagadougou
Quito
Rio de Janeiro
Shanghai
Sydney
Tehran
Tianjin
Xiamen
Yinchuan


In [20]:
for index, city in enumerate(city_options[2:],start=0):

  city_name = city.text
  if(city_name=='Luanda'):
    continue
  print(f"Clicking on {city_name}")

  # 点击城市
  city.click()
  
  ##############################################################
  # generate city definition
  ##############################################################
  # press the generate city definition buttons
  generate_city_definition_T2_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(1) > div')[0]
  generate_city_definition_T1_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(2) > div')[0]
  generate_city_definition_T2_element.click()
  generate_city_definition_T1_element.click()

  #give the appliciton some time to generate the definitions
  time.sleep(20)

  ##############################################################
  # compute indicators
  ##############################################################
  #click the compute button
  compute_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(4) > div > div:nth-child(4) > div > div > div > div.ui-widget.ui-button > div')[0]
  compute_element.click()
  time.sleep(5)

  ##############################################################
  # collect the indicators
  ##############################################################
  # go through each chart in a new window
  pop_out_icons = driver.find_elements(By.CLASS_NAME,"pop-out-icon")  
  print(len(pop_out_icons))
  #click the pop put icon
  for icon in pop_out_icons:
    icon.click()

  # Assuming now have four windows (the original one and the new three)
  window_handles = driver.window_handles
  print(window_handles)

  # Switch to the new window or tab
  new_window_handle = [handle for handle in window_handles if handle != original_window_handle]

  for i,w in enumerate(new_window_handle):
    driver.switch_to.window(w)   
    cityname=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr > td:nth-child(1)')
    LCPC=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')
    CIUI=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')
    
    # different charts
     #primary
    if cityname:
      print('Print values with sleep')
      values=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr>td')
      keys=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > thead > tr>th')      
      for k,v in zip(keys,values):
        df.at[index,k.text]=v.text

    else:
      #LCPC
      if 'Land Consumption per Capita' in LCPC[0].text:  
        rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
        rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
        print(len(rectangles))
        for r in rectangles:
        # 使用ActionChains进行悬停操作
          actions = ActionChains(driver)  
          actions.move_to_element(r).perform()
          # 等待一段时间
          time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
          texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7) > g > g > text')
          print(len(texts))
          for t in texts:
            print(t.text)
          if len(texts)>0:
            df.at[index,texts[1].text[:-1]+' '+texts[0].text]=texts[2].text
          
      #CIUI
      # if "Change in Urban Infill" in CIUI[0].text:
      else:
        rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
        print(len(rectangles))
        for r in rectangles:
        # 使用ActionChains进行悬停操作
          actions = ActionChains(driver)  
          actions.move_to_element(r).perform()
          # 等待一段时间
          time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
          texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7)> g > g > text')
          print(len(texts))
          for t in texts:
            print(t.text)
          if len(texts)>0:
            df.at[index,texts[1].text[:-1]]=texts[2].text
    time.sleep(3)

  
  # 关闭所有新窗口
  for w in new_window_handle:      
      driver.switch_to.window(w)
      driver.close()
  #swith to the origin main window
  driver.switch_to.window(original_window_handle) 
  city_element.click()
  

# 保存DataFrame到Excel文件
df.to_excel(save_file_path, index=False)
# 关闭浏览器窗口
driver.quit()

Clicking on Stockholm
3


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x009972A3+45731]
	(No symbol) [0x00922D51]
	(No symbol) [0x008186D0]
	(No symbol) [0x0084CAC4]
	(No symbol) [0x00844FC0]
	(No symbol) [0x00867FDC]
	(No symbol) [0x00844A4E]
	(No symbol) [0x00868254]
	(No symbol) [0x0087B7A2]
	(No symbol) [0x00867DD6]
	(No symbol) [0x008431F6]
	(No symbol) [0x0084439D]
	GetHandleVerifier [0x00CA0716+3229462]
	GetHandleVerifier [0x00CE84C8+3523784]
	GetHandleVerifier [0x00CE214C+3498316]
	GetHandleVerifier [0x00A21680+611968]
	(No symbol) [0x0092CCCC]
	(No symbol) [0x00928DF8]
	(No symbol) [0x00928F1D]
	(No symbol) [0x0091B2C7]
	BaseThreadInitThunk [0x771D7BA9+25]
	RtlInitializeExceptionChain [0x77EABD2B+107]
	RtlClearBits [0x77EABCAF+191]


In [252]:
df

,City,Interval,LCR,PGR,LCRPGR,City Definition data,Built-up data,Population data,Within city definition T1 Built-up (km²),"Within city definition T1 Population (*10,000)",Within city definition T1 LCPC (m²/person),Within city definition T2 Built-up (km²),"Within city definition T2 Population (*10,000)",Within city definition T2 LCPC (m²/person),% Change in LCPC LCPC (m²/person),Built-up T1,Built-up T2,% Change in urban infill
0,Stockholm,1990-2015,0.007,0.014,0.506,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,"2,131.64","2,593.298",82.198,300.063,201.928,148.599,-16.221,"2,131.64","4,022.641",88.711
1,Gothenburg,1990-2015,0.005,0.009,0.573,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,149.869,63.947,234.365,182.294,84.789,214.998,-8.264,149.869,168.301,12.299
2,Abu Dhabi,1990-2015,0.008,0.047,0.171,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,64.583,40.918,157.835,98.217,165.229,59.443,-62.338,64.583,70.864,9.725
3,Accra,1990-2015,0.019,0.034,0.554,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,512.261,186.176,275.149,843.58,470.865,179.156,-34.888,512.261,718.967,40.352
4,Batam,1990-2015,0.025,0.071,0.36,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,31.363,15.657,200.309,83.191,129.113,64.432,-67.833,31.363,44.41,41.601
5,Beijing,1990-2015,0.008,0.035,0.24,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,"1,515.037",691.114,219.217,"2,861.539","2,277.11",125.665,-42.675,"1,515.037","1,640.908",8.308
6,Buenos Aires,1990-2015,0.007,0.013,0.534,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,"1,431.884","1,060.769",134.985,"1,902.682","1,558.96",122.048,-9.584,"1,431.884","1,634.226",14.131
7,Cairo,1990-2015,0.006,0.017,0.375,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,663.007,"1,166.728",56.826,876.336,"1,917.888",45.693,-19.592,663.007,744.11,12.233
8,Charleston,1990-2015,0.015,0.02,0.732,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,162.918,26.093,624.374,258.859,48.307,535.862,-14.176,162.918,239.264,46.861
9,Dar Es Salaam,1990-2015,0.027,0.059,0.463,GHS-Pop 1km 2015,GHS-Built 38m,GHS-Pop 250m,197.779,125.052,158.157,403.929,579.194,69.74,-55.905,197.779,371.334,87.752


In [254]:
# 保存DataFrame到Excel文件
df.to_excel("E:/AG2421/sdg_results.xlsx", index=False)

# # 关闭浏览器窗口
# driver.quit()

In [21]:
  generate_city_definition_T2_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(1) > div')[0]
  generate_city_definition_T1_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(2) > div')[0]
  generate_city_definition_T2_element.click()
  generate_city_definition_T1_element.click()

In [28]:
pop_out_icons = driver.find_elements(By.CLASS_NAME,"pop-out-icon") [0] 
# print(len(pop_out_icons))
pop_out_icons.click()
# for icon in pop_out_icons:
#   icon.click()

In [114]:
  window_handles = driver.window_handles
print(window_handles)

['1126FFA8303D6D00E996440F23A263E7', '5116DD2B6E1F5D1CAC4B0BEB90B18DFE', '197DC261E17CF3B7ED0CEBD59C47AAC3', '9F70FAD48799BD01510B780D5557C453']


In [115]:
  new_window_handle = [handle for handle in window_handles if handle != original_window_handle]
print(new_window_handle)

['5116DD2B6E1F5D1CAC4B0BEB90B18DFE', '197DC261E17CF3B7ED0CEBD59C47AAC3', '9F70FAD48799BD01510B780D5557C453']


In [255]:
  driver.switch_to.window(original_window_handle) 

In [192]:
driver.switch_to.window(new_window_handle[2])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00AE72A3+45731]
	(No symbol) [0x00A72D51]
	(No symbol) [0x0096880D]
	(No symbol) [0x0094F75E]
	(No symbol) [0x009BC11B]
	(No symbol) [0x009C04FB]
	(No symbol) [0x009B7FA3]
	(No symbol) [0x009931F6]
	(No symbol) [0x0099439D]
	GetHandleVerifier [0x00DF0716+3229462]
	GetHandleVerifier [0x00E384C8+3523784]
	GetHandleVerifier [0x00E3214C+3498316]
	GetHandleVerifier [0x00B71680+611968]
	(No symbol) [0x00A7CCCC]
	(No symbol) [0x00A78DF8]
	(No symbol) [0x00A78F1D]
	(No symbol) [0x00A6B2C7]
	BaseThreadInitThunk [0x768C7BA9+25]
	RtlInitializeExceptionChain [0x773ABD2B+107]
	RtlClearBits [0x773ABCAF+191]


In [171]:
cityname=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr > td:nth-child(1)')

In [204]:

for i,w in enumerate(new_window_handle):
  driver.switch_to.window(w)  
  cityname=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr > td:nth-child(1)')
  LCPC=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')
  CIUI=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')

  print(LCPC)
      
  if cityname:
    print('Print values with sleep')
    values=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr>td')
    keys=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > thead > tr>th')      
    for k,v in zip(keys,values):
      print(k.text,v.text)  
  else:      
    #LCPC
    if 'Land Consumption per Capita' in LCPC[0].text:  
      rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
    #   print(len(rectangles))
    #   for r in rectangles:
    #   # 使用ActionChains进行悬停操作
    #     actions = ActionChains(driver)  
    #     actions.move_to_element(r).perform()
    #     # 等待一段时间
    #     time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
    #     texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7) > g > g > text')
    #     print(len(texts))
    #     for t in texts:
    #       print(t.text)
    #     df.at[index,texts[1].text+' '+texts[0].text]=texts[2].text
    else:
      rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
      print(len(rectangles))
      for r in rectangles:
      # 使用ActionChains进行悬停操作
        actions = ActionChains(driver)  
        actions.move_to_element(r).perform()
        # 等待一段时间
        time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
        texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7)> g > g > text')
        print(len(texts))
        for t in texts:
          print(t.text)
        df.at[index,texts[1].text[:-1]]=texts[2].text

  time.sleep(2)

  #chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7) > g > g:nth-child(2) > text
  #chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)



# # 关闭所有新窗口
# for w in new_window_handle:      
#     driver.switch_to.window(w)
#     driver.close()
  
  

[<selenium.webdriver.remote.webelement.WebElement (session="d66310ab2ca64967c02134f7962b0566", element="D98F2465988911412307B1C8883EF7EC_element_22427")>]
3
3
Stockholm
Built-up T1:
238.572
3
Stockholm
Built-up T2:
282.011
3
Stockholm
% Change in urban infill :
18.208
[]
Print values with sleep
City Stockholm
Interval 1990-2015
LCR 0.007
PGR 0.014
LCRPGR 0.506
City Definition data GHS-Pop 1km 2015
Built-up data GHS-Built 38m
Population data GHS-Pop 250m
[<selenium.webdriver.remote.webelement.WebElement (session="d66310ab2ca64967c02134f7962b0566", element="4CBB4BE69CE5A66ADA71850E697FBD95_element_22033")>]


In [183]:
  rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')

In [184]:

actions = ActionChains(driver)  
actions.move_to_element(rectangles[0]).perform()

In [185]:
texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7) > g > g > text')

In [186]:

for t in texts:
  print(t.text)

Stockholm
Built-up T1:
238.572


In [190]:
df['Built-up T1']

1    NaN
0    NaN
Name: Built-up T1, dtype: object